In [40]:
import os
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('default')
import pandas as pd
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower
plt.tight_layout()
import matplotlib 
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 20}
matplotlib.rc('font', **font)

<Figure size 1200x800 with 0 Axes>

In [ ]:
# ==================================================
# HÀM 1: CHUYỂN ĐỔI TEXT THÀNH SỐ (ENCODING)
# ==================================================
def changeTextFeatureToNumeric(df, cols):
    """
    Chuyển đổi các cột có giá trị text thành số
    VD: 'MALE' -> 0, 'FEMALE' -> 1
        'Adelie' -> 0, 'Chinstrap' -> 1, 'Gentoo' -> 2
    
    Args:
        df: DataFrame cần xử lý
        cols: Danh sách tên các cột cần chuyển đổi
        
    Returns:
        DataFrame với các cột text đã được chuyển thành số
    """
    for col in cols:
        try:
            # pd.factorize(): Chuyển mỗi giá trị unique thành 1 số (0, 1, 2, ...)
            # sort=False: Giữ nguyên thứ tự xuất hiện, không sắp xếp
            a = df[[col]].apply(lambda col: pd.factorize(col, sort=False)[0])[col]
            
            # Loại bỏ cột text cũ
            b = df.drop([col], axis=1)
            
            # Gộp lại: DataFrame mới = cột cũ (đã chuyển thành số) + các cột khác
            df = pd.concat([b, a], axis=1, join='inner')
        except:
            pass  # Nếu lỗi (VD: cột không tồn tại), bỏ qua
    return df

# ==================================================
# HÀM 2: CHIA DỮ LIỆU THÀNH TRAIN/VALIDATION/TEST
# ==================================================
def splitTrainValidTest(df, percentage, data_name):
    """
    Chia dữ liệu thành 3 tập: TRAIN / VALIDATION / TEST
    
    Args:
        df: DataFrame đã được encode (text -> số)
        percentage: [train_ratio, test_start_ratio]
                    VD: [0.5, 0.7] nghĩa là:
                    - 0-50%: TRAIN
                    - 50-70%: VALIDATION  
                    - 70-100%: TEST
        data_name: Tên thư mục để lưu kết quả
        
    Returns:
        df_train, df_valid, df_test
        
    Ví dụ: percentage = [0.5, 0.7]
    - Train: 50% đầu
    - Validation: 20% giữa (50% -> 70%)
    - Test: 30% cuối (70% -> 100%)
    """
    # Chia theo loài (Species đã được encode thành 0, 1, 2)
    df_Adelie = df[df['Species'] == 0]      # Adelie
    df_Chinstrap = df[df['Species'] == 1]   # Chinstrap
    df_Gentoo = df[df['Species'] == 2]      # Gentoo

    # Shuffle từng loài
    df_Adelie = df_Adelie.sample(frac=1)
    df_Chinstrap = df_Chinstrap.sample(frac=1)
    df_Gentoo = df_Gentoo.sample(frac=1)
    
    # -------- Hàm con: Lấy phần TRAIN --------
    def dfGetTrainData(df):
        df_len = len(df)
        # Lấy từ đầu đến percentage[0] (VD: 0-50%)
        return df.head(int(df_len * percentage[0]))

    # -------- Hàm con: Lấy phần VALIDATION --------
    def dfGetValidData(df):
        df_len = len(df)
        # Lấy từ percentage[0] đến percentage[1] (VD: 50-70%)
        return df[int(df_len * percentage[0]):int(df_len * percentage[1])]
    
    # -------- Hàm con: Lấy phần TEST --------
    def dfGetTestData(df):
        df_len = len(df)
        # Lấy từ percentage[1] đến cuối (VD: 70-100%)
        return df[int(df_len * percentage[1]):int(df_len)]
    
    # Chia train cho từng loài
    df_Adelie_train = dfGetTrainData(df_Adelie)
    df_Chinstrap_train = dfGetTrainData(df_Chinstrap)
    df_Gentoo_train = dfGetTrainData(df_Gentoo)
    
    # Chia validation cho từng loài
    df_Adelie_valid = dfGetValidData(df_Adelie)
    df_Chinstrap_valid = dfGetValidData(df_Chinstrap)
    df_Gentoo_valid = dfGetValidData(df_Gentoo)

    # Chia test cho từng loài
    df_Adelie_test = dfGetTestData(df_Adelie)
    df_Chinstrap_test = dfGetTestData(df_Chinstrap)
    df_Gentoo_test = dfGetTestData(df_Gentoo)
    
    # Gộp và shuffle
    frames = [df_Adelie_train, df_Chinstrap_train, df_Gentoo_train]
    df_train = pd.concat(frames).sample(frac=1)
    
    frames = [df_Adelie_valid, df_Chinstrap_valid, df_Gentoo_valid]
    df_valid = pd.concat(frames).sample(frac=1)
    
    frames = [df_Adelie_test, df_Chinstrap_test, df_Gentoo_test]
    df_test = pd.concat(frames).sample(frac=1)
    
    # Tạo thư mục và lưu file
    try:
        os.makedirs('../data/' + data_name)
    except OSError as e:
        pass
    
    df_train.to_csv('../data/' + data_name + '/train_data.csv', index = False)
    df_valid.to_csv('../data/' + data_name + '/valid_data.csv', index = False)
    df_test.to_csv('../data/' + data_name + '/test_data.csv', index = False)
    
    return df_train, df_valid, df_test

# ==================================================
# HÀM 3: CHIA TRAIN/TEST (Không có Validation)
# ==================================================
def splitTrainTest(df, percentage, data_name):
    """
    Giống như hàm splitTrainValidTest nhưng chỉ chia 2 tập: Train/Test
    
    Args:
        percentage: Tỷ lệ cho train (VD: 0.8 = 80% train, 20% test)
    """
    df_Adelie = df[df['Species'] == 0]
    df_Chinstrap = df[df['Species'] == 1]
    df_Gentoo = df[df['Species'] == 2]

    # Shuffle
    df_Adelie = df_Adelie.sample(frac=1)
    df_Chinstrap = df_Chinstrap.sample(frac=1)
    df_Gentoo = df_Gentoo.sample(frac=1)
    
    def dfGetTrainData(df, percentage):
        df_len = len(df)
        return df.head(int(df_len * percentage))

    def dfGetTestData(df, percentage):
        df_len = len(df)    
        return df.tail(df_len - int(df_len * percentage))
    
    df_Adelie_train = dfGetTrainData(df_Adelie, percentage)
    df_Chinstrap_train = dfGetTrainData(df_Chinstrap, percentage)
    df_Gentoo_train = dfGetTrainData(df_Gentoo, percentage)

    df_Adelie_test = dfGetTestData(df_Adelie, percentage)
    df_Chinstrap_test = dfGetTestData(df_Chinstrap, percentage)
    df_Gentoo_test = dfGetTestData(df_Gentoo, percentage)
    
    frames = [df_Adelie_train, df_Chinstrap_train, df_Gentoo_train]
    df_train = pd.concat(frames).sample(frac=1)
    frames = [df_Adelie_test, df_Chinstrap_test, df_Gentoo_test]
    df_test = pd.concat(frames).sample(frac=1)
    
    try:
        os.makedirs('../data/' + data_name)
    except OSError as e:
        pass
    
    df_train.to_csv('../data/' + data_name + '/train_data.csv', index = False)
    df_test.to_csv('../data/' + data_name + '/test_data.csv', index = False)
    
# ==================================================
# HÀM 4: LẤY MẪU CON TỪ TẬP TRAIN (SUB-SAMPLING)
# ==================================================
def getTrainSamples(df, size, source, data_name):
    """
    Tạo tập train nhỏ hơn với số lượng mẫu cố định
    Mục đích: Thử nghiệm model với lượng dữ liệu khác nhau
              VD: 50, 80, 110, 140, 170 mẫu
    
    Args:
        df: DataFrame train gốc
        size: Số lượng mẫu muốn lấy (VD: 50, 110, 170)
        source: Thư mục nguồn
        data_name: Tên file đầu ra
        
    Returns:
        DataFrame với số lượng mẫu = size
    """
    len_df = len(df)
    percentage = size / len(df)  # Tính tỷ lệ phần trăm tương ứng
    
    # Chia theo loài
    df_Adelie = df[df['Species'] == 0]
    df_Chinstrap = df[df['Species'] == 1]
    df_Gentoo = df[df['Species'] == 2]

    # Shuffle
    df_Adelie = df_Adelie.sample(frac=1)
    df_Chinstrap = df_Chinstrap.sample(frac=1)
    df_Gentoo = df_Gentoo.sample(frac=1)
    
    # Tính số mẫu cho từng loài (giữ tỷ lệ)
    size_Adelie = int(len(df_Adelie) * percentage)
    size_Chinstrap = int(len(df_Chinstrap) * percentage)
    size_Gentoo = int(len(df_Gentoo) * percentage)
    
    def dfGetTrainData(df, size):
        return df.head(size)
    
    # Lấy số lượng mẫu tương ứng từ mỗi loài
    df_Adelie_train = dfGetTrainData(df_Adelie, size_Adelie)
    # Chinstrap lấy phần còn lại để đủ tổng = size
    df_Chinstrap_train = dfGetTrainData(df_Chinstrap, int(size) - size_Adelie - size_Gentoo)
    df_Gentoo_train = dfGetTrainData(df_Gentoo, size_Gentoo)
    
    # Gộp và shuffle
    frames = [df_Adelie_train, df_Chinstrap_train, df_Gentoo_train]
    df_train = pd.concat(frames).sample(frac=1)
    
    # Lưu với tên file chứa số lượng mẫu
    try:
        os.makedirs('../data/' + source)
    except OSError as e:
        pass
    
    df_train.to_csv('../data/' + source + '/train_data_' + str(size) + '.csv', index = False)
    
    return df_train

In [ ]:
# ==================================================
# CHẠY QUY TRÌNH CHIA DỮ LIỆU TRAIN/VALIDATION/TEST
# ==================================================
save_folder = 'train_0.5_valid_0.7_test'

# Đọc dữ liệu đã loại bỏ outliers
df_remove_outliers = pd.read_csv("../data/data_remove_outliers.csv")

# Chuyển đổi text thành số cho các cột:
# - 'Clutch Completion': Trạng thái ấp trứng
# - 'Sex': Giới tính (MALE/FEMALE)
# - 'Species': Loài chim (Adelie/Chinstrap/Gentoo)
# - 'a': Cột bổ sung (nếu có)
df_remove_outliers = changeTextFeatureToNumeric(df_remove_outliers, ['Clutch Completion', 'Sex', 'Species', 'a'])

# LƯU Ý: Dòng dưới đây bị comment out (không chạy)
# df_remove_outliers = df_remove_outliers.fillna(df_remove_outliers.mean())
# splitTrainTest(df_remove_outliers, percentage = 0.7, data_name = save_folder)

# Chia dữ liệu thành 3 tập với tỷ lệ:
# - 0-50%: TRAIN (50%)
# - 50-70%: VALIDATION (20%)
# - 70-100%: TEST (30%)
a, b, c = splitTrainValidTest(df_remove_outliers, percentage = [0.5, 0.7], data_name = save_folder)

# Hiển thị DataFrame đã encode
df_remove_outliers

,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Body Mass (g),Delta 15 N (o/oo),Delta 13 C (o/oo),Sex,Species
0,39.1,18.7,181.0,3750.0,NaN,NaN,0,0
1,39.5,17.4,186.0,3800.0,8.94956,-24.69454,1,0
2,40.3,18.0,195.0,3250.0,8.36821,-25.33302,1,0
3,36.7,19.3,193.0,3450.0,8.76651,-25.32426,1,0
4,39.3,20.6,190.0,3650.0,8.66496,-25.29805,0,0
...,...,...,...,...,...,...,...,...
337,47.2,13.7,214.0,4925.0,7.99184,-26.20538,1,2
338,46.8,14.3,215.0,4850.0,8.41151,-26.13832,1,2
339,50.4,15.7,222.0,5750.0,8.30166,-26.04117,0,2
340,45.2,14.8,212.0,5200.0,8.24246,-26.11969,1,2


In [ ]:
# ==================================================
# KIỂM TRA SỐ LƯỢNG MẪU TRONG TẬP TRAIN
# ==================================================
a = pd.read_csv("../data/train_0.5_valid_0.7_test/train_data.csv")
len(a)  # Hiển thị tổng số mẫu trong tập train

170

In [ ]:
# ==================================================
# TẠO CÁC TẬP TRAIN NHỎ HƠN (SUB-SAMPLING)
# ==================================================
# Tạo 5 tập train với số lượng mẫu khác nhau:
# - 50 mẫu
# - 80 mẫu  
# - 110 mẫu
# - 140 mẫu
# - 170 mẫu
# Mục đích: Thử nghiệm hiệu suất model với lượng dữ liệu training khác nhau
for i in range(50, 180, 30):  # range(50, 180, 30) = [50, 80, 110, 140, 170]
    getTrainSamples(a, i, 'train_0.5_valid_0.7_test', 'a')

# Tackle NaN

In [ ]:
# ==================================================
# HÀM XỬ LÝ GIÁ TRỊ NaN - THAY BẰNG MEAN
# ==================================================
def fillNaN(save_folder, file_name):
    """
    Xử lý giá trị thiếu (NaN) bằng cách thay bằng giá trị trung bình
    
    Phương pháp: Mean Imputation
    - Với mỗi cột có NaN, tính giá trị trung bình (mean) của cột đó
    - Thay tất cả NaN trong cột bằng giá trị mean
    
    Ưu điểm: Đơn giản, không làm thay đổi phân phối tổng thể
    Nhược điểm: Giảm độ phân tán (variance) của dữ liệu
    
    Args:
        save_folder: Thư mục chứa file
        file_name: Tên file cần xử lý (không có đuôi .csv)
    """
    # Đọc file gốc
    df = pd.read_csv('../data/' + save_folder + '/' + file_name + '.csv')
    
    # fillna(mean()): Thay NaN bằng giá trị trung bình của mỗi cột
    df = df.fillna(df.mean())
    
    # Lưu file mới với hậu tố '_NaNmean'
    df.to_csv('../data/' + save_folder + '/' + file_name + '_NaNmean.csv', index = False)

In [ ]:
# ==================================================
# ÁP DỤNG XỬ LÝ NaN CHO TẤT CẢ CÁC TẬP TRAIN
# ==================================================
# Xử lý NaN cho các tập train có kích thước khác nhau (50, 80, 110, 140, 170)
for i in range(50, 180, 30):
    fillNaN('train_0.5_valid_0.7_test', 'train_data_' + str(i))

In [ ]:
# ==================================================
# ÁP DỤNG XỬ LÝ NaN CHO TẬP TEST VÀ VALIDATION
# ==================================================
fillNaN('train_0.5_valid_0.7_test', 'test_data')
fillNaN('train_0.5_valid_0.7_test', 'valid_data')

# Normalization

In [ ]:
# ==================================================
# HÀM CHUẨN HÓA DỮ LIỆU (NORMALIZATION/STANDARDIZATION)
# ==================================================
def normalizationTrainTest(save_folder, file_name):
    """
    Chuẩn hóa dữ liệu sử dụng Z-score standardization
    
    Công thức: z = (x - mean) / (std + epsilon)
    - x: Giá trị gốc
    - mean: Giá trị trung bình
    - std: Độ lệch chuẩn (standard deviation)
    - epsilon: Số rất nhỏ để tránh chia cho 0
    
    SAU CHUẨN HÓA: Dữ liệu có mean=0, std=1
    
    ⚠️ QUAN TRỌNG: 
    - Tính mean và std từ TẬP TRAIN
    - Áp dụng mean và std này cho cả VALIDATION và TEST
    - KHÔNG tính riêng mean/std cho validation/test (gây data leakage)
    
    Args:
        save_folder: Thư mục chứa file
        file_name: Tên file train cần chuẩn hóa
    """
    eps=1e-9  # Epsilon = 0.000000001 (tránh chia cho 0)
    
    # Đọc tập train (đã xử lý NaN)
    df = pd.read_csv('../data/' + save_folder + '/' + file_name + '_NaNmean.csv')
    
    # Tính mean và std từ TẬP TRAIN
    train_data_mean = np.mean(df, axis=0)  # Mean của mỗi cột
    train_data_mean['Species'] = 1  # Không chuẩn hóa cột Species (giữ nguyên 0,1,2)
    
    train_data_std = np.std(df, axis=0)  # Độ lệch chuẩn của mỗi cột
    train_data_std['Species'] = 1  # Không chuẩn hóa cột Species
    
    # Chuẩn hóa tập TRAIN
    df = (df - train_data_mean) / (train_data_std + eps)
    df.to_csv('../data/' + save_folder + '/' + file_name + '_Normalization.csv', index = False)
    
    # Chuẩn hóa tập TEST (dùng mean/std từ TRAIN)
    df_test = pd.read_csv('../data/' + save_folder + '/' + 'test_data_NaNmean' + '.csv')
    df_test = (df_test - train_data_mean) / (train_data_std + eps)
    df_test.to_csv('../data/' + save_folder + '/' + file_name + '_test.csv', index = False)
    
    # Chuẩn hóa tập VALIDATION (dùng mean/std từ TRAIN)
    df_valid = pd.read_csv('../data/' + save_folder + '/' + 'valid_data_NaNmean' + '.csv')
    df_valid = (df_valid - train_data_mean) / (train_data_std + eps)
    df_valid.to_csv('../data/' + save_folder + '/' + file_name + '_valid.csv', index = False)

In [ ]:
# ==================================================
# ÁP DỤNG CHUẨN HÓA CHO TẤT CẢ CÁC TẬP TRAIN
# ==================================================
# Chuẩn hóa các tập train với kích thước khác nhau (50, 80, 110, 140, 170)
# và cả tập validation, test tương ứng
for i in range(50, 180, 30):
    normalizationTrainTest('train_0.5_valid_0.7_test', 'train_data_' + str(i))

In [ ]:
# ==================================================
# ĐỌC MỘT TẬP TRAIN ĐỂ KIỂM TRA
# ==================================================
# Đọc tập train 170 mẫu (chưa xử lý NaN và chuẩn hóa)
df = pd.read_csv('../data/' + 'train_0.5_valid_0.7_test' + '/' + 'train_data_170' + '.csv')

In [ ]:
# ==================================================
# KIỂM TRA GIÁ TRỊ MEAN VÀ STD
# ==================================================
# Tính mean và std của tập train (để hiểu quá trình chuẩn hóa)
train_data_mean = np.mean(df, axis=0)
train_data_mean['Species'] = 0  # Không chuẩn hóa Species

train_data_std = np.std(df, axis=0)
train_data_std['Species'] = 1  # Không chuẩn hóa Species

# Hiển thị mean và std
train_data_mean, train_data_std

(Culmen Length (mm)       44.164706
 Culmen Depth (mm)        17.223529
 Flipper Length (mm)     201.435294
 Body Mass (g)          4233.382353
 Delta 15 N (o/oo)         8.734083
 Delta 13 C (o/oo)       -25.706497
 Sex                       0.452941
 Species                   0.000000
 dtype: float64,
 Culmen Length (mm)       5.384011
 Culmen Depth (mm)        1.952383
 Flipper Length (mm)     14.181966
 Body Mass (g)          798.403481
 Delta 15 N (o/oo)        0.527354
 Delta 13 C (o/oo)        0.803437
 Sex                      0.542996
 Species                  1.000000
 dtype: float64)